In [8]:
import pandas as pd
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [9]:
market = Market()
sec = SEC()
fin = ADatabase("earnings")
umod = UniversalModeler()

In [10]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [11]:
factors = ['assets',
 'liabilitiesandstockholdersequity',
 'incometaxexpensebenefit',
 'retainedearningsaccumulateddeficit',
 'accumulatedothercomprehensiveincomelossnetoftax',
 'earningspersharebasic',
 'earningspersharediluted',
 'propertyplantandequipmentnet',
 'cashandcashequivalentsatcarryingvalue',
 'entitycommonstocksharesoutstanding',
 'weightedaveragenumberofdilutedsharesoutstanding',
 'weightedaveragenumberofsharesoutstandingbasic',
 'stockholdersequity']

In [12]:
included_columns=[
    "year",
    "quarter",
    "ticker",
    "adjclose",
    "y"
]

In [13]:
included_columns.extend(factors)

In [14]:
market.connect()
sec.connect()
training_sets = []
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        cik = int(sp500[sp500["ticker"]==ticker]["CIK"])
        prices = market.retrieve_ticker_prices("prices",ticker)
        prices = p.column_date_processing(prices)
        prices["year"] = [x.year for x in prices["date"]]
        prices["quarter"] = [x.quarter for x in prices["date"]]
        filing = sec.retrieve_filing_data(cik)
        filing = p.column_date_processing(filing)
        filing = filing.groupby(["year","quarter"]).mean().reset_index()
        ticker_data = prices.copy()
        ticker_data.sort_values("date",ascending=True,inplace=True)
        ticker_data["adjclose"] = [float(x) for x in ticker_data["adjclose"]]
        ticker_data = ticker_data.groupby(["year","quarter"]).mean().reset_index()
        ticker_data.dropna(inplace=True)
        ticker_data["ticker"] = ticker
        ticker_data = ticker_data.merge(filing,on=["year","quarter"],how="left").reset_index()
        ticker_data["y"] = ticker_data["earningspersharebasic"].shift(-4)
        ticker_data = ticker_data[included_columns]
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()
sec.disconnect()

  1%|██                                                                                                                                                                         | 6/490 [00:01<02:04,  3.90it/s]

"['entitycommonstocksharesoutstanding'] not in index"


  2%|██▊                                                                                                                                                                        | 8/490 [00:02<02:25,  3.31it/s]

"['stockholdersequity'] not in index"


  4%|██████▌                                                                                                                                                                   | 19/490 [00:06<02:57,  2.65it/s]

...

 97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍     | 474/490 [02:54<00:08,  1.97it/s]

"['propertyplantandequipmentnet'] not in index"


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 489/490 [03:00<00:00,  1.87it/s]

"['cashandcashequivalentsatcarryingvalue'] not in index"


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [03:00<00:00,  2.71it/s]


In [15]:
data = pd.concat(training_sets)

In [16]:
data = data.dropna()

In [17]:
data

,year,quarter,ticker,adjclose,y,assets,liabilitiesandstockholdersequity,incometaxexpensebenefit,retainedearningsaccumulateddeficit,accumulatedothercomprehensiveincomelossnetoftax,earningspersharebasic,earningspersharediluted,propertyplantandequipmentnet,cashandcashequivalentsatcarryingvalue,entitycommonstocksharesoutstanding,weightedaveragenumberofdilutedsharesoutstanding,weightedaveragenumberofsharesoutstandingbasic,stockholdersequity
48,2012,1,MMM,62.705929,2.783636,2.967400e+10,3.088600e+10,1.551333e+09,2.717150e+10,-4.284000e+09,2.555455,2.519091,7.472500e+09,2.621250e+09,694543763.0,7.170667e+08,7.075667e+08,1.554150e+10
49,2012,2,MMM,62.787037,2.201818,3.181550e+10,3.181550e+10,4.520000e+08,2.860300e+10,-4.905000e+09,1.565000,1.540000,7.710500e+09,2.504800e+09,693872048.0,7.162500e+08,7.041500e+08,1.580100e+10
50,2012,3,MMM,66.710493,2.522500,3.247750e+10,3.247750e+10,6.990000e+08,2.890650e+10,-5.018500e+09,2.440000,2.400000,7.686000e+09,3.070000e+09,691319463.0,7.149750e+08,7.039250e+08,1.591300e+10
51,2012,4,MMM,67.131673,3.410000,3.304750e+10,3.304750e+10,9.145000e+08,2.924900e+10,-4.786500e+09,3.217500,3.172500,7.802500e+09,3.000250e+09,691931278.0,7.113250e+08,7.015750e+08,1.653200e+10
52,2013,1,MMM,75.359447,2.955455,3.188267e+10,3.274600e+10,1.702000e+09,2.951350e+10,-4.887500e+09,2.783636,2.743636,8.022000e+09,2.879750e+09,689990255.0,7.159333e+08,7.053667e+08,1.649750e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,2020,3,ZTS,150.821274,1.450000,1.230700e+10,1.230700e+10,1.307500e+08,4.737500e+09,-7.910000e+08,1.167500,1.160000,1.975500e+09,2.643500e+09,475144704.0,4.804250e+08,4.771750e+08,2.845000e+09
31,2020,4,ZTS,159.798699,2.070000,1.263500e+10,1.263500e+10,1.897500e+08,4.976500e+09,-7.305000e+08,1.735000,1.720000,2.019500e+09,2.794000e+09,475267819.0,4.802000e+08,4.768750e+08,3.155000e+09
33,2021,2,ZTS,171.169916,1.220000,1.370250e+10,1.370250e+10,1.015000e+08,5.879000e+09,-6.815000e+08,1.035000,1.025000,2.214500e+09,3.603000e+09,474766701.0,4.784500e+08,4.755500e+08,3.929000e+09
34,2021,3,ZTS,199.153900,1.712500,1.383900e+10,1.383900e+10,1.662500e+08,6.075500e+09,-6.765000e+08,1.450000,1.442500,2.244000e+09,3.631000e+09,473944193.0,4.778000e+08,4.751750e+08,4.059500e+09


In [18]:
fin.connect()
years = 4
for modeler in [umod]:
    for year in tqdm(range(2016,2023)):
        training_slice = data[(data["year"]<year) & (data["year"] >= year - years)].reset_index(drop=True)
        prediction_set = data[data["year"]==year].reset_index(drop=True)
        set_name = "sim"
        stuff = modeler.model(training_slice,prediction_set,factors,tf=False)
        stuff["training_years"] = years
        fin.store(set_name,stuff)
fin.disconnect()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:18<00:00, 11.17s/it]


In [19]:
stuff

,year,quarter,ticker,adjclose,y,assets,liabilitiesandstockholdersequity,incometaxexpensebenefit,retainedearningsaccumulateddeficit,accumulatedothercomprehensiveincomelossnetoftax,...,entitycommonstocksharesoutstanding,weightedaveragenumberofdilutedsharesoutstanding,weightedaveragenumberofsharesoutstandingbasic,stockholdersequity,xgb_prediction,xgb_score,cat_prediction,cat_score,prediction,training_years
0,2022,1,ABT,121.161350,3.476667,7.387200e+10,7.387200e+10,6.756667e+08,2.957750e+10,-8.595000e+09,...,1.763482e+09,1.785333e+09,1.772000e+09,3.429300e+10,2.808381,0.659322,-6854.765698,-0.001547,-3425.978658,4
1,2022,1,ABBV,138.456888,4.312500,1.485470e+11,1.485470e+11,2.533333e+08,2.091000e+09,-3.008000e+09,...,1.768754e+09,1.644667e+09,1.639333e+09,1.424200e+10,3.489667,0.659322,113609.381941,-0.001547,56806.435804,4
2,2022,1,ATVI,77.442590,2.753333,2.408250e+10,2.408250e+10,3.380000e+08,1.085800e+10,-6.000000e+08,...,7.792349e+08,7.776667e+08,7.716667e+08,1.420825e+10,2.488876,0.659322,-86874.956921,-0.001547,-43436.234022,4
3,2022,1,ADBE,481.100806,6.545000,2.618550e+10,2.618550e+10,1.210833e+08,2.309550e+10,-1.522500e+08,...,4.721000e+08,4.825917e+08,4.786000e+08,1.291688e+10,6.533318,0.659322,-56402.802442,-0.001547,-28198.134562,4
4,2022,1,ADP,209.187251,2.692500,5.393755e+10,5.393755e+10,2.912500e+08,1.973145e+10,-1.589000e+08,...,4.200453e+08,4.258500e+08,4.239500e+08,5.588583e+09,2.538036,0.659322,9373.473027,-0.001547,4688.005531,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,2022,1,GWW,485.266612,21.013333,6.443500e+09,6.443500e+09,2.923333e+08,9.139500e+09,-7.850000e+07,...,5.110790e+07,5.360000e+07,5.336667e+07,1.851000e+09,20.977245,0.659322,-24807.922401,-0.001547,-12393.472578,4
258,2022,1,WYNN,84.264677,-9.913333,8.093191e+09,1.320019e+10,2.473283e+08,-1.910249e+09,4.804000e+06,...,1.158987e+08,1.091633e+08,1.090833e+08,-2.832075e+08,-3.984015,0.659322,18854.147504,-0.001547,9425.081745,4
259,2022,1,XYL,94.415087,1.916667,8.245333e+09,8.513000e+09,4.333333e+07,2.042000e+09,-3.920000e+08,...,1.799011e+08,1.812737e+08,1.801167e+08,3.093000e+09,2.167966,0.659322,-10226.520601,-0.001547,-5112.176318,4
260,2022,1,ZBRA,459.417581,11.316667,5.795000e+09,5.795000e+09,8.033333e+07,3.154500e+09,-4.900000e+07,...,5.307974e+07,5.413670e+07,5.362634e+07,2.075500e+09,13.642515,0.659322,29738.914664,-0.001547,14876.278590,4
